# Merged dataset generator

### Run this file to generate the merged datasets from the cleaned files.
### At the bottom of the file, you can find correlations
You can copy that script or check it out on your own. Generating correlations for a big dataset may take some time, but they are done after the files are created, so you can safely ignore the long running time if you are not interested



`You must run the generate_clean_data.ipynb before using this. If you make changes to the raw data/processing, run that again before running this.`


In [27]:
import pandas as pd
from sklearn.impute import SimpleImputer
import os
from sklearn.preprocessing import StandardScaler

### Generating merged datasets

`status on datasets`
[X] used [!] lbocked [ ] unused

In [28]:
# RAW		CLEANED		USED
# 11		11			9

#LABELS:
# RAW		CLEANED		USED
# 2			2			2

# [X] breda_population
# [X] education_levels
# [X] labor_force
# [!] livability
# [X] migration
# [X] public_nuisance_criminal
# [X] public_nuisance_noncriminal
# [X] police_response_times
# [X] weather_metrics
# [!] workplaces_info
# [X] bencheslightpoi

# [X] crime_metrics_simplified
# [X] crime_metrics

In [29]:
if len(os.listdir('../Datasets/cleaned')) == 0:
  raise Exception("YOU MUST RUN /DatasetCleaning/generate_clean_data.ipynb SUCCESSFULLY FIRST!\n\nThe file will not run without the cleaned data files.") 

In [30]:
#time series
df_nuisance_criminal = pd.read_csv('../Datasets/cleaned/monthly_12_23_breda_nuisancereportscriminal_bytypecount.csv')
df_nuisance_noncriminal = pd.read_csv('../Datasets/cleaned/monthly_12_23_breda_nuisancereportsnoncriminal_bytypecount.csv')
df_response_times = pd.read_csv('../Datasets/cleaned/monthly_15_23_city_responsetimes_bylenghtcount.csv')

df_livability = pd.read_csv('../Datasets/cleaned/monthly_12_23_breda_livability.csv')
df_migration_breda_change = pd.read_csv('../Datasets/cleaned/monthly_12_23_breda_migrantpop.csv')

#cross-sectional
df_education = pd.read_csv('../Datasets/cleaned/2021_breda_educationlevel_bylevelbypopulation.csv')
df_labor = pd.read_csv('../Datasets/cleaned/2021_breda_laborparticipation_bynetbycontracttype.csv')
df_population = pd.read_csv('../Datasets/cleaned/2021_breda_censusstats.csv')
df_poi = pd.read_csv('../Datasets/cleaned/2021_breda_bencheslightspoi.csv')

#misc
df_weather = pd.read_csv('../Datasets/cleaned/2021_weather_bymonthlyavg.csv')

#label
df_label_crime_bytype = pd.read_csv('../Datasets/cleaned/monthly_12_23_breda_crimestats_bytypecount.csv')
df_label_crime_aggregate = pd.read_csv('../Datasets/cleaned/monthly_12_23_breda_crimestats_aggregate.csv')

# merging a little bit
df_merged_time_series = pd.merge(df_nuisance_criminal, df_nuisance_noncriminal, on=['Year', 'Month', 'NeighbourhoodCode'], how='outer')

df_migration_breda_change

df_merged_time_series = pd.merge(df_merged_time_series, df_livability, on=['Year', 'Month', 'NeighbourhoodCode'], how='outer')
df_merged_time_series = pd.merge(df_merged_time_series, df_migration_breda_change, on=['Year', 'Month', 'NeighbourhoodCode'], how='outer')


Imputating some missing data (specifically, from 2012 to 2015)

In [31]:
def addMissingYearRows(df):
	years_range = range(2012, 2015)
	months_range = range(1, 13)
	all_combinations = [(year, month) for year in years_range for month in months_range]

	new_rows = pd.DataFrame(all_combinations, columns=["Year", "Month"])
	merged_df = new_rows.merge(df, on=["Year", "Month"], how="left")
	return merged_df

def addMissingMonthsOnly(df):
	months_range = range(1, 13)
	all_combinations = [(month) for month in months_range]
	new_rows = pd.DataFrame(all_combinations, columns=["Month"])
	merged = new_rows.merge(df, on = ["Month"], how = "left")
	return merged

#based on label; make sure neighbourhoods are complete
unique_neighborhoods = df_label_crime_bytype['NeighbourhoodCode'].unique()
def addMissingNeighbourhoods(df):
	missing_df = pd.DataFrame({'NeighbourhoodCode': unique_neighborhoods})
	return pd.merge(df, missing_df, on='NeighbourhoodCode', how='outer')



In [32]:
imputer = SimpleImputer(strategy='median')


merged_df = None
if True:
    merged_df = addMissingYearRows(df_response_times)
else:
	years_range = range(2012, 2015)
	months_range = range(1, 13)
	all_combinations = [(year, month) for year in years_range for month in months_range]

	new_rows = pd.DataFrame(all_combinations, columns=["Year", "Month"])
	merged_df = new_rows.merge(df_response_times, on=["Year", "Month"], how="left")

numerical_cols = ["ResponseTimeScore", "SlowResponseTimePenalty"]

#imputed = imputer.fit_transform(merged_df[numerical_cols], df_response_times[num])


print(df_response_times.head())
#print(imputed)

#merged_df[numerical_cols] = imputed
median_values = df_response_times[numerical_cols].median()
merged_df = merged_df.fillna(median_values)
merged_df.sort_values(["Year", "Month"], inplace=True)
merged_df.reset_index(drop=True, inplace=True)

merged_df = merged_df.astype(int)

   Year  Month  ResponseTimeScore  SlowResponseTimePenalty
0  2015      1               0.93                     1.00
1  2015      2               0.90                     1.70
2  2015      3               0.94                     1.10
3  2015      4               0.94                     0.80
4  2015      5               0.89                     2.30


We need to calculate some of the missing numbers from the labor participation and education data.

In [33]:
df_merged_responses = pd.concat([merged_df, df_response_times])
#we merge the reponse times (Year, Month) and the time series (Year, Month, NeighbourhoodCode)
df_merged_timebased = pd.merge(df_merged_responses, df_merged_time_series, on=['Year', 'Month'], how='right')

In [34]:
df_labor = addMissingNeighbourhoods(df_labor)

numerical_cols = ["NetLaborParticipation","FlexibleContracts","SelfContract"]
df_labor[numerical_cols] = imputer.fit_transform(df_labor[numerical_cols])
df_labor.reset_index(drop=True, inplace=True)

df_labor[numerical_cols] = df_labor[numerical_cols].astype(int)

##and education
df_education = addMissingNeighbourhoods(df_education)
numerical_cols = ["PopulationEduLow","PopulationEduMedium","PopulationEduHigh"]

df_education[numerical_cols] = imputer.fit_transform(df_education[numerical_cols])
df_education.reset_index(drop=True, inplace=True)

df_education[numerical_cols] = df_education[numerical_cols].astype(int)


##oh points of interest
df_poi = addMissingNeighbourhoods(df_poi)
numerical_cols = ["Benches","Lights","POI"]

df_poi[numerical_cols] = imputer.fit_transform(df_poi[numerical_cols])
df_poi.reset_index(drop=True, inplace=True)

df_poi[numerical_cols] = df_poi[numerical_cols].astype(int)


In [35]:
#Let's add relevant data to neighbourhoods
df_merged_neighbourhoods = pd.merge(df_labor, df_education, 					on=['NeighbourhoodCode'], how='left')
df_merged_neighbourhoods = pd.merge(df_merged_neighbourhoods, df_population, 	on=['NeighbourhoodCode'], how="left")
df_merged_neighbourhoods = pd.merge(df_merged_neighbourhoods, df_poi, 			on=['NeighbourhoodCode'], how="left")

#Combine the two
df_final = pd.merge(df_merged_timebased, df_merged_neighbourhoods, on=['NeighbourhoodCode'], how='outer')

col_nbhc = df_final.pop("NeighbourhoodCode")
df_final.insert(0, "NeighbourhoodCode", col_nbhc)
df_final.sort_values(["NeighbourhoodCode", "Year", "Month"], inplace=True)

In [36]:
#We can try to do this:
if False:
    df_label_crime_total['CrimeRate'] = (df_label_crime_total["CrimeCount"] / df_final["Inhabitants"]) * 100000
    df_label_crime_total.drop(['CrimeCount'], axis=1, inplace=True)

In [37]:
## ADDING GROUPS ##
CRIMETYPEGROUPS = []
def add_group(group_name, members):
    CRIMETYPEGROUPS.append({'name': group_name, 'members': members})
def convert_to_dict():
    groups_dict = {}
    for group in CRIMETYPEGROUPS:
        group_name = group['name']
        members = group['members']
        groups_dict[group_name] = {member: True for member in members}
    return groups_dict
def check_membership(member):
    if member == "Total felonies":
        return None
    for group in CRIMETYPEGROUPS:
        group_name = group['name']
        members = group['members']
        if member.strip() in members:
            return group_name
    print("No group found for member: '" + member + "', returning None")
    return None

add_group('Property', ["Theft motor vehicles", "Theft from public transport vehicles", "Thefts (water)",'Theft/burglary of companies, etc.', "Theft/burglary box/garage/shed",'Theft of mopeds, mopeds and bicycles', "Theft from/from/from ov vehicles","Theft from/from motor vehicles","Shoplifting", "Home theft/burglary", "Motor Vehicle Theft", "Other property crimes", "Pickpocketing"])
add_group('PropertyDamage', ["Fire/Explosion",  "Destruction or property damage",])
add_group('Violent', ["Threat", "Domestic Violation", 'Murder, Manslaughter', "Open violence (person)", "Robbery", "Street robbery"])
add_group('White-collar', ["Vertical Fraud", "Fraud (other)", "Horizontal Fraud"])
add_group('Drug-related', ["Drug trafficking", "Drugs/drink nuisance"])
add_group('Cyber', ["Cybercrime"])
add_group('Hate', ["Discrimination"])
add_group('Juvenile', ["Encroachment on public order", "Fireworks"])
add_group('Organized', ["Arms Trade", "Human trafficking", "People smuggling"])
add_group('VehicleOps', ["Water", "Under the influence (away)", "Under the influence (air)", "Under the influence (water)", "Air (other)", "Under the influence (road)"])
add_group('Neglect', ["Waste", "Transport of hazardous substances", "Structure of the Environmental Management Act", "Abuse", "Accidents (road)", "Food safety", "Pesticides", "Road (other)"])
add_group('noidea????', ["Total felonies", "Special Laws", "Spatial planning", "Soil", "Quality of life (other)", "Animals", 'Building materials',  "Immigration care", "Most", "Nature and landscape", "Neighbor rumor (relationship problems)"])

GROUPS_DICT = convert_to_dict()

In [38]:

df = df_label_crime_bytype

non_member_columns = ["NeighbourhoodCode", "Year", "Month"]
member_columns = [col for col in df.columns if col not in non_member_columns]

grouped_df = pd.DataFrame()
grouped_df[non_member_columns] = df[non_member_columns]

grouped_columns = {}
for column in member_columns:
    group = check_membership(column)
    if group is None:
        continue
    if group not in grouped_columns:
        grouped_columns[group] = []
    grouped_columns[group].append(column)

for group, columns in grouped_columns.items():
    grouped_df[group] = df[columns].sum(axis=1)
grouped_df

,NeighbourhoodCode,Year,Month,Neglect,Juvenile,White-collar,Organized,noidea????,Cyber,PropertyDamage,Hate,Violent,Drug-related,Property,VehicleOps
0,Bavel,2012,1,7.00,0.00,0.00,0.00,0.00,0.00,7.00,0.00,4.00,0.00,30.00,0.00
1,Bavel,2012,2,9.00,0.00,3.00,0.00,0.00,0.00,5.00,0.00,0.00,0.00,27.00,2.00
2,Bavel,2012,3,2.00,0.00,1.00,0.00,0.00,0.00,3.00,0.00,5.00,0.00,35.00,0.00
3,Bavel,2012,4,5.00,0.00,0.00,0.00,0.00,1.00,2.00,0.00,2.00,1.00,25.00,0.00
4,Bavel,2012,5,0.00,0.00,1.00,1.00,0.00,1.00,2.00,0.00,1.00,0.00,16.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7255,Zandberg,2022,8,5.00,0.00,2.00,0.00,0.00,0.00,2.00,0.00,2.00,0.00,6.00,2.00
7256,Zandberg,2022,9,3.00,0.00,2.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,16.00,1.00
7257,Zandberg,2022,10,5.00,0.00,2.00,0.00,0.00,0.00,2.00,0.00,1.00,0.00,12.00,1.00
7258,Zandberg,2022,11,4.00,0.00,4.00,0.00,0.00,2.00,2.00,0.00,0.00,0.00,10.00,0.00


In [39]:
#nice labels datasets
if False:
	col_crimerate = df_label_crime_total.pop('CrimeRate')
	df_label_crime_total.insert(3, "CrimeRate", col_crimerate)

In [40]:
#Get rid of all unknown label neighbourhoods
if False:
	valid_neighbourhoods = df_label_crime_bytype['NeighbourhoodCode'].unique()
	df_final = df_final[df_final['NeighbourhoodCode'].isin(valid_neighbourhoods)]

In [41]:
#weather (ideally done once all extensions are done)
df_final = pd.merge(df_final, df_weather, on="Month")
df_final.sort_values(["NeighbourhoodCode", "Year", "Month"], inplace=True)

In [42]:
#create multiple approaches to missing data
b_HandleMissingValues = True

if b_HandleMissingValues:
    #fill with mean
    df_final_filledmean = df_final.fillna(df_final.mean())
    #leave empty
    df_final_empty = df_final.copy()
    #remove
    df_final_delrows = df_final.dropna()

C:\Users\benjm\AppData\Local\Temp\ipykernel_21856\3306406288.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_final_filledmean = df_final.fillna(df_final.mean())


In [43]:
#fancy crime metrics

df_crimes_fancy_metrics = grouped_df.copy().drop(['Neglect', 'Juvenile', 'White-collar', 'Organized', 'noidea????', 'Cyber', 'Hate', 'VehicleOps', 'Drug-related', 'PropertyDamage'], axis=1)

#per total crimes
total_p_crimes = grouped_df.groupby(['Year', 'Month'])['Property'].sum() 
total_v_crimes = grouped_df.groupby(['Year', 'Month'])['Violent'].sum()
total_burglary = df_label_crime_bytype.groupby(['Year', 'Month'])['Theft/burglary box/garage/shed'].sum() + df_label_crime_bytype.groupby(['Year', 'Month'])['Theft/burglary of companies, etc.'].sum()
total_burglary = pd.Series(total_burglary, name="Burglaries_total")
total_allcrimes = df_label_crime_bytype.groupby(['Year', 'Month'])['Total felonies'].sum()
total_allcrimes = pd.Series(total_allcrimes, name="AllCrimes_total")

df_burglary_sep = pd.DataFrame({
    'NeighbourhoodCode': df_label_crime_bytype['NeighbourhoodCode'],
    'Year': df_label_crime_bytype['Year'],
    'Month': df_label_crime_bytype['Month'],
    'Burglary': df_label_crime_bytype['Theft/burglary box/garage/shed'] + df_label_crime_bytype['Theft/burglary of companies, etc.'],
    'TotalCrimes': df_label_crime_bytype['Total felonies'],
})


df_crimes_fancy_metrics = df_crimes_fancy_metrics.merge(total_p_crimes, on=['Year', 'Month'], suffixes=['', '_total'])
df_crimes_fancy_metrics = df_crimes_fancy_metrics.merge(total_v_crimes, on=['Year', 'Month'], suffixes=['', '_total'])
df_crimes_fancy_metrics = df_crimes_fancy_metrics.merge(total_burglary, on=['Year', 'Month'], suffixes=['', '_total'])
df_crimes_fancy_metrics = df_crimes_fancy_metrics.merge(total_allcrimes, on=['Year', 'Month'], suffixes=['', '_total'])
df_crimes_fancy_metrics = df_crimes_fancy_metrics.merge(df_burglary_sep, on=['NeighbourhoodCode', 'Year', 'Month'])

df_crimes_fancy_metrics['PropertyCrimesPercentInCity'] = (df_crimes_fancy_metrics['Property'] / df_crimes_fancy_metrics['Property_total']) * 100
df_crimes_fancy_metrics['ViolentCrimesPercentInCity'] = (df_crimes_fancy_metrics['Violent'] / df_crimes_fancy_metrics['Violent_total']) * 100

#per inhabitants
total_inhabitants = df_final_filledmean.groupby(['Year', 'Month'])['Inhabitants'].mean()
df_crimes_fancy_metrics = df_crimes_fancy_metrics.merge(total_inhabitants, on=['Year', 'Month'], suffixes=['', ''])
df_crimes_fancy_metrics['PropertyCrimesPerThousandInhabitants'] = (df_crimes_fancy_metrics['Property'] / df_crimes_fancy_metrics['Inhabitants'] * 1000)
df_crimes_fancy_metrics['ViolentCrimesPerThousandInhabitants'] = (df_crimes_fancy_metrics['Violent'] / df_crimes_fancy_metrics['Inhabitants'] * 1000)

#burglary
df_crimes_fancy_metrics['BurglaryCrimesPercentInCity'] = (df_crimes_fancy_metrics['Burglary'] / df_crimes_fancy_metrics['Burglaries_total']) * 100
df_crimes_fancy_metrics['BurglaryCrimesPerThousandInhabitants'] = (df_crimes_fancy_metrics['Burglary'] / df_crimes_fancy_metrics['Inhabitants'] * 1000)

#all crimes
df_crimes_fancy_metrics['AllCrimesPercentInCity'] = (df_crimes_fancy_metrics['TotalCrimes'] / df_crimes_fancy_metrics['AllCrimes_total']) * 100
df_crimes_fancy_metrics['AllCrimesPerThousandInhabitants'] = (df_crimes_fancy_metrics['TotalCrimes'] / df_crimes_fancy_metrics['Inhabitants'] * 1000)


df_crime_metrics = pd.DataFrame({
	"NeighbourhoodCode": grouped_df["NeighbourhoodCode"],
	"Year": grouped_df["Year"],
	"Month": grouped_df["Month"],
    "PropertyCrimesCount": grouped_df["Property"],
    "PropertyCrimesPerThousandInhabitants": df_crimes_fancy_metrics['PropertyCrimesPerThousandInhabitants'],
    "PropertyCrimesPercentInCity": df_crimes_fancy_metrics['PropertyCrimesPercentInCity'],
    "ViolentCrimesCount": grouped_df["Violent"],
    "ViolentCrimesPerThousandInhabitants": df_crimes_fancy_metrics['ViolentCrimesPerThousandInhabitants'],
    "ViolentCrimesPercentInCity": df_crimes_fancy_metrics['ViolentCrimesPercentInCity'],
    "BurglariesCount": df_crimes_fancy_metrics['Burglary'],
    "BurglaryCrimesPerThousandInhabitants": df_crimes_fancy_metrics['BurglaryCrimesPerThousandInhabitants'],
    "BurglaryCrimesPercentInCity": df_crimes_fancy_metrics['BurglaryCrimesPercentInCity'],
    "AllCrimeCount": df_crimes_fancy_metrics['TotalCrimes'],
    "AllCrimesPerThousandInhabitants": df_crimes_fancy_metrics['AllCrimesPerThousandInhabitants'],
    "AllCrimesPercentInCity": df_crimes_fancy_metrics['AllCrimesPercentInCity'],
}).round(4)


In [44]:
datasetCheckList=None
columns = ['NeighbourhoodCode', 'Year', 'Month']
exception = False
if b_HandleMissingValues:
    datasetCheckList = (df_final_filledmean, df_final_empty, df_final_delrows)
else:
    datasetCheckList = (df_final_filledmean, df_final_empty, df_final_delrows)

for df in datasetCheckList:
    for column in columns:
        value_counts = df[column].value_counts()
        unique_counts = value_counts.unique()
        if len(unique_counts) > 1:
            print(f"The value count in column '{column}' is different from the others:")
            print(value_counts)
            print()
            print("database the error was found in:", df)
            exception = True

if exception:
    raise Exception("Merging integrity error! Check the log.")


#if didn't raise error, build indexes
for dataset in datasetCheckList:
    dataset.set_index(columns, inplace=True)
    dataset.sort_index(inplace = True)
    print("Number of missing values in dataset:", dataset.isnull().sum().sum())

Number of missing values in dataset: 0
Number of missing values in dataset: 0
Number of missing values in dataset: 0


In [45]:
#standardization
def standard(df, column_str:str):
	col = df[column_str]	
	min = col.min()
	max = col.max()
	df[column_str] = (col - min) / (max - min)


exclude_cols = ['NeighbourhoodCode', 'Year', 'Month']
cols_to_standardize = [col for col in df_final.columns if col not in exclude_cols]

if False:
	scaler = StandardScaler()

	if b_HandleMissingValues:
		df_final_filledmean[cols_to_standardize] = scaler.fit_transform(df_final_filledmean[cols_to_standardize])
		df_final_empty[cols_to_standardize] = scaler.fit_transform(df_final_empty[cols_to_standardize])
		df_final_delrows[cols_to_standardize] = scaler.fit_transform(df_final_delrows[cols_to_standardize])

		df_final_filledmean[cols_to_standardize]
	else:
		df_final[cols_to_standardize] = scaler.fit_transform(df_final[cols_to_standardize])

if True:
	if b_HandleMissingValues:
		for column in cols_to_standardize:
			standard(df_final_filledmean, column)
			standard(df_final_empty, column)
			standard(df_final_delrows, column)
	else:
		for column in cols_to_standardize:
			standard(df_final, column)


In [46]:
if b_HandleMissingValues:
    df_final_filledmean.drop_duplicates(inplace=True)
    df_final_empty.drop_duplicates(inplace=True)
    df_final_delrows.drop_duplicates(inplace=True)

In [47]:
#to file
#df_label_crime_total.to_csv('../Datasets/finalized/label_crime_total_perinhabitant.csv', index=False)
df_label_crime_bytype.to_csv('../Datasets/finalized/label_crime_bytype.csv', index=False)
grouped_df.to_csv('../Datasets/finalized/label_crime_bygroup.csv', index=False)
df_crime_metrics.to_csv('../Datasets/finalized/label_crime_allmetrics.csv', index=False)

if b_HandleMissingValues:
    df_final_filledmean.to_csv('../Datasets/finalized/features_missing_filledmean.csv')
    df_final_empty.to_csv('../Datasets/finalized/features_all_data.csv')
    df_final_delrows.to_csv('../Datasets/finalized/features_complete_only.csv')
else:
    df_final.to_csv('../Datasets/finalized/features_all_data.csv', index=False)

### Correlations

In [48]:
from scipy.stats import spearmanr

In [49]:
# Set 'NeighbourhoodCode' as the index for both dataframes
df_final.set_index('NeighbourhoodCode', inplace=True)
#df_label_crime_total.set_index('NeighbourhoodCode', inplace=True)

# Merge data and label dataframes
merged_df = pd.merge(df_final, df_label_crime_aggregate, on=['Year', 'Month'])

Linear correlation

In [50]:



# Assuming you have a DataFrame called 'data' with features and the label
correlation_matrix = merged_df.corr()

# Get the correlation between features and the label
label_correlation = correlation_matrix['CrimeRatePerPop']

# Sort the correlations in descending order
sorted_correlations = label_correlation.sort_values(ascending=False)

# Set the Pandas options for display format
pd.set_option('display.float_format', lambda x: f'{x:.2f}')



In [51]:
#better matrix
features = merged_df

# Calculate the correlation matrix
corr_matrix = features.corr()

columns = correlation_matrix.columns
n = len(columns)

# Create an empty DataFrame to store the non-zero correlations
non_zero_df = pd.DataFrame(columns=['Variable 1', 'Variable 2', 'Correlation'])

# Iterate over the upper triangular part of the correlation matrix
for i in range(n):
    for j in range(i+1, n):
        variable1 = columns[i]
        variable2 = columns[j]
        correlation = correlation_matrix.iloc[i, j]
        
        # Append the non-zero correlations to the DataFrame
        if correlation != 0:
            non_zero_df = non_zero_df.append({'Variable 1': variable1, 'Variable 2': variable2, 'Correlation': correlation}, ignore_index=True)

# Sort the DataFrame by correlation in descending order
non_zero_df = non_zero_df.sort_values('Correlation', ascending=False)

C:\Users\benjm\AppData\Local\Temp\ipykernel_21856\2743594204.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  non_zero_df = non_zero_df.append({'Variable 1': variable1, 'Variable 2': variable2, 'Correlation': correlation}, ignore_index=True)
C:\Users\benjm\AppData\Local\Temp\ipykernel_21856\2743594204.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  non_zero_df = non_zero_df.append({'Variable 1': variable1, 'Variable 2': variable2, 'Correlation': correlation}, ignore_index=True)
C:\Users\benjm\AppData\Local\Temp\ipykernel_21856\2743594204.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  non_zero_df = non_zero_df.append({'Variable 1': variable1, 'Variable 2': variable2, 'Correlation': correlation}, ignore_index=T

Spearman correlation

In [52]:

features = merged_df.drop('CrimeRatePerPop', axis=1)
label = merged_df['CrimeRatePerPop']

# Compute Spearman's rank-order correlation for each feature
correlation_results = []
for column in features.columns:
    correlation, p_value = spearmanr(features[column], label)
    correlation_results.append({'Feature': column, 'Correlation': correlation, 'p-value': p_value})

# Create a new dataframe to store the correlation results
correlation_df = pd.DataFrame(correlation_results)

# Sort the dataframe by absolute correlation values
correlation_df['Absolute Correlation'] = correlation_df['Correlation'].abs()
correlation_df = correlation_df.sort_values('Absolute Correlation', ascending=False)


c:\Users\benjm\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "


In [53]:
#print correlations:

print("Sorted correlations:\n", sorted_correlations)

print("\n\nSpearman correlation:\n", correlation_df)

print("\n\nsorted variable to variable correlations:\n")
non_zero_df

Sorted correlations:
 CrimeRatePerPop                        1.00
NightC                                 0.27
Month                                  0.24
DayC                                   0.23
Rainfall(mm)                           0.21
Rain Days                              0.12
SlowResponseTimePenalty                0.10
Year                                   0.10
Daylight hours                         0.06
Registered Crimes                      0.03
NuisanceReportsNonCriminal             0.02
Migrated                               0.02
fys                                    0.01
lbm                                    0.00
afw                                    0.00
onv                                    0.00
won                                    0.00
ResponseTimeScore                      0.00
vrz                                    0.00
soc                                    0.00
y15-25%                                0.00
NetLaborParticipation                  0.00
FlexibleCo

,Variable 1,Variable 2,Correlation
243,lbm,afw,1.00
925,DayC,NightC,0.97
438,vrz,UrbanityLevel,0.90
928,DayC,Daylight hours,0.88
281,afw,onv,0.86
...,...,...,...
548,FlexibleContracts,Averagepeopleperhousehold,-0.81
399,soc,Populationdensitykm2,-0.82
406,soc,UrbanityLevel,-0.83
759,Averagepeopleperhousehold,Rentalproperies(%),-0.85
